# Toy Neural Net

[Neural Network Simulator](https://www.mladdict.com/neural-network-simulator)

## Testing
* following [this guide](https://selenium-python.readthedocs.io/index.html)
    * [Chapter 4](https://selenium-python.readthedocs.io/locating-elements.html) explains how to locate elements (I am using CSS selector)
* this repository uses [ChromeDriver](https://sites.google.com/a/chromium.org/chromedriver/downloads) for Linux (v77)

In [54]:
# Libraries

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Python
import re

# ____________________________________________________________________________

# Setup Driver / Launch Site
driver = webdriver.Chrome("./chromedriver")
driver.get("https://www.mladdict.com/neural-network-simulator")


# Data Collection 
def test_vals_collect():
    
    def get_value(css_selector):
        return float(
            re.findall(r'[-+]?\d*\.\d+|\d+',
                       driver.find_element_by_css_selector(css_selector).text
                      )[0]
        )

    # 0) Initialization
    
    selector_parent_path = '#app > div > div:nth-child(2) > div > div > div:nth-child(2) > div > div:nth-child(1) > div.col-md-10.col-sm-12.col-xs-12.vcenter > div > svg > svg:nth-child(2) >'
    
    w_2_11 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(1) > svg > text')
    w_2_21 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(4) > svg > text')
    w_2_12 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(2) > svg > text')
    w_2_22 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(5) > svg > text')
    w_2_13 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(3) > svg > text')
    w_2_23 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(6) > svg > text')

    w_3_11 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(1) > svg > text')
    w_3_21 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(2) > svg > text')
    w_3_31 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(3) > svg > text')

    # 1) Input Layer
    
    x_1 = 0
    x_2 = 0

    # 2) Hidden Layer

    # 3) Output Layer

    # 4) Calculate the cost

    # 5) Error in the output Layer

    # 6) Error in the hidden layer

    # 7) Calculate the error with respect to weights between hidden and output layer

    # 8) Calculate the error with respect to weights between input and hidden layer

    # 9) Update the weights between hidden and output layer

    # 10) Update the weights between input and hidden layer
    
    
    return 0 # object? (what would be easiest for writting to a csv)

test_vals_collect()



# End
driver.close()

In [ ]:
# Input
x_1 = np.array([1, 0, 1, 0])
x_2 = np.array([1, 1, 0, 0])
y   = np.array([0, 1, 1, 0])

# Weights (Hidden Layer 1)
W_1_1 = 0.09979296
W_2_1 = 0.00725321

W_1_2 = -0.68081849
W_2_2 = -0.73768875

W_1_3 = 0.02592901
W_2_3 = 0.0495334

# Weights (Output)
W_1_1 = -0.85672409
W_2_1 = 0.64333851
W_3_1 = 0.00054628